In [39]:
%run 2gis_functions.ipynb

Ready: 2023-04-02 23:41:31


In [4]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [40]:
@show_time
def count_users_requests(file_path):
    year = file_path.split('_')[-2]
    requests_name = f'Запросы {year}'
    users_name = f'Пользователи {year}'
    df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
    cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
    cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
    user_cnt_year = df.user.nunique().compute()
    print(year, user_cnt_year)
    cnt_user_grouped_year = (cnt_req_grouped
                  .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                  .rename(columns={'date':requests_name, 'user':users_name})
                  .sort_values(requests_name, ascending=False)
                  .fillna(0)
                  .astype(int)
                  )
    return cnt_user_grouped_year

@show_time
def count_total_users(*files):
    df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
    df_all = dd.concat(df_list)
    cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
    cnt_user_total = df_all.user.nunique().compute()
    print(cnt_user_total)
    return cnt_user_grouped_total

@show_time
def make_base_counted_file(*files):
    df = count_users_requests(files[0])
    df.to_csv(f'{files[0]}_count_users_requests.csv', single_file=True)
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName')
        df.to_csv(f'{file}_count_users_requests.csv', single_file=True)
        
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName')
    df.to_csv('{}_count_total_users.csv'.format(file.split("\\")[-2]), single_file=True)
    return df


@show_time
def count_users_requests(file_path):
    try:
        cnt_user_grouped_year = dd.read_csv(f'{file_path}_count_users_requests.csv')
        print('read ' + f'{file_path}_count_users_requests.csv')
    except:
        year = file_path.split('_')[-2]
        requests_name = f'Запросы {year}'
        users_name = f'Пользователи {year}'
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
        cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
        #user_cnt_year = df.user.nunique().compute()
        #print(year, user_cnt_year)
        cnt_user_grouped_year = (cnt_req_grouped
                      .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                      .rename(columns={'date':requests_name, 'user':users_name})
                      .sort_values(requests_name, ascending=False)
                      .fillna(0)
                      .astype(int)
                      )
        cnt_user_grouped_year.to_csv(f'{file_path}_count_users_requests.csv', single_file=True)
    finally:
        df = dd.read_csv(f'{file_path}_count_users_requests.csv')
    return df

@show_time
def count_total_users(*files):
    file_path = "\\".join(files[0][0].split("\\")[:-1]) + "\\" + files[0][0].split("\\")[-2]
    try:
        cnt_user_grouped_total = dd.read_csv("{}_count_total_users.csv".format(file_path))
        print('read ' + "{}_count_total_users.csv".format(file_path))
    except:
        df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
        df_all = dd.concat(df_list)
        cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
        #cnt_user_total = df_all.user.nunique().compute()
        #print(cnt_user_total)
        cnt_user_grouped_total.to_csv(f'{file_path}_count_total_users.csv', single_file=True)
    finally:
        df = dd.read_csv("{}_count_total_users.csv".format(file_path))
    return df

@show_time
def make_base_counted_file(*files):
    df = count_users_requests(files[0])
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName', how='outer')
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName', how='outer')
    df.to_csv('master_file.csv', single_file=True)
    return df

from dask.diagnostics import ProgressBar
ProgressBar().register()

In [ ]:
x = make_base_counted_file(
"D:\файлы_2гис_2023\Астана\Астана_2020_975007.csv",
"D:\файлы_2гис_2023\Астана\Астана_2021_975009.csv",
"D:\файлы_2гис_2023\Астана\Астана_2022_975011.csv",
"D:\файлы_2гис_2023\Астана\Астана_2023_975039.csv",

)

[########################################] | 100% Completed | 11m 42s
[########################################] | 100% Completed | 11m 42s
[########################################] | 100% Completed | 11m 42s
Start:     2023-04-02 23:41:37
End:       2023-04-02 23:53:20
Duration:  0:11:42
[########################################] | 100% Completed | 11m 41s
[########################################] | 100% Completed | 11m 41s
[########################################] | 100% Completed | 11m 41s
Start:     2023-04-02 23:53:20
End:       2023-04-03 00:05:01
Duration:  0:11:41
[########################################] | 100% Completed | 15m 58s
[########################################] | 100% Completed | 15m 58s
[########################################] | 100% Completed | 15m 58s
Start:     2023-04-03 00:05:01
End:       2023-04-03 00:21:00
Duration:  0:15:58
[########################################] | 100% Completed | 25m 19s
[########################################] | 100% Complet